# Identify Field Management Zones from a Collection of NDVI Images nad Soil Moisture Surveys

In this tutorial you will learn how to:

- Find, access, and download products from Google Earth Engine
- Compute NDVI from image bands 
- Load custom field boundaries in vector format
- Plot raster files
- Stack raster files to generate space-time arrays
- Cluster multiple images to generate field management zones
- Export map of resulting field management zones as prescription maps in Shapefile format
- Load points of soil moisture data collected in the field
- Create raster for field
- Interpolate points to raster
- Identify drier and wetter than average zones


Before you start, make sure you ran the *setup* notebook and have your GEE account.

You can find the *setup* notebook in the link: https://github.com/soilwater/precisionag-workshop-2024/blob/main/setup_python_geospatial_analysis.ipynb

### Import necessary modules

In [ ]:
# Import modules
import ee
import glob
import requests
import matplotlib.pyplot as plt
from matplotlib import colors
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import json
from datetime import datetime, timedelta
from scipy.ndimage import gaussian_filter
from scipy.interpolate import griddata, RBFInterpolator


### Initialize Google Earth Engine

In [ ]:
# Authenticate
ee.Authenticate()

# Initialize API
ee.Initialize()

### Create helper functions

In [ ]:
def save_gee_to_geotiff(ee_image, filename, crs, scale, geom, bands=[]):
    """
    Function to save images from Google Earth Engine into local hard drive.
    """
    image_url = ee_image.getDownloadUrl({'region': geom,'scale':scale, 
                                         'bands': bands,
                                         'crs': f'EPSG:{crs}', 
                                         'format': 'GEO_TIFF'})
    
    # Request data using URL and save data as a new GeoTiff file
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
            print(f"Saved image {filename}")
    else:
        print("Failed to download image")


def array_to_df(arr):
    """Function to convert list into dataframe"""
    df = pd.DataFrame(arr[1:])
    df.columns = arr[0]
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    return df


In [ ]:
# Create our own colormap
hex_palette = ['#FEFEFE','#CE7E45', '#DF923D', '#F1B555', '#FCD163', '#99B718', '#74A901',
             '#66A000', '#529400', '#3E8601', '#207401', '#056201', '#004C00', '#023B01',
             '#012E01', '#011D01', '#011301']


# Use the built-in ListedColormap function to do the conversion


### Define area of interest

We will use a cropland field within the Konza Prairie Biological Station near Manhattan, KS. The field has a site of the National Ecological Observatory Network that you can also use to retrieve additional data.

In [ ]:
# Import boundary for area of interest (aoi)
#with open('neon_kona_bnd.geojson') as file:
#    aoi_json = json.load(file)

# Read field boundary with Geopandas


# Get JSON format (.to_json() gives us a string, so we use the json module to create a proposer json object

# Define the ee.Geometry

# Create mask for field


In [ ]:
# Get centroid so that we can define a point

# Get centroids from GEE geometry



### Get NDVI timeseries to inspect growing season trend

In [ ]:
# Define start and end dates



In [ ]:
# Define point of interest


In [ ]:
# Get collection for Modis 16-day


In [ ]:
# Convert array into dataframe


In [ ]:
# Create figure to visualize time series


### Download Sentinel data

In [ ]:
# Re-define start and end dates based on time series


In [ ]:
# Select Sentinel 2 data

# Get the list of available image dates



In [ ]:
# Select all or a subset of the collection dates

# Download each image



In [ ]:
# Read one image to inspect data

# Get number of rows and columns for later use

# Visualize map for specific date



In [ ]:
# Read all image names


In [ ]:
# Load all images


In [ ]:
# Convert to array


In [ ]:
# Plot all of them


### Access and select data within DataArray

In [ ]:
# Plot one image


In [ ]:
# Only areas with specific NDVI


In [ ]:
# Plot temporal NDVI

### Compute relative difference

For each NDVI layer we will normalize the values by first subtracting the mean, and then dividing by the mean. This was we will obtain a new grid showing areas of the field that have more or less biomass than teh field average.

$$ RD = \frac{NDVI - \overline{NDVI}}{\overline{NDVI}}$$


In [ ]:
# Create function to calculate relative difference


In [ ]:
# Compute relative difference


In [ ]:
# Compute mean relative difference



In [ ]:
# Plot mean relative difference



### Find management zones

We will use clustering analysis to find homogeneous management zones.

In [ ]:
# Import modules



In [ ]:
# Input data using all surveys
clustering_input_data = rel_diff.values.reshape(-1, len(selected_dates))

# Input clustering data is the resulting mean relative difference
#clustering_input_data = field_mrd.values.reshape(-1, 1)


In [ ]:
# Cluster 

In [ ]:
# Plot management zones


### Vectorize resulting raster management zones

In [ ]:
# Import modules


In [ ]:
# Create polygons from each pixel

# Create a GeoDataFrame with the polygons and their cluster labels


In [ ]:
# Plot


In [ ]:
# Export shapefile as prescription map


## Inspect soil moisture surveys

In [ ]:
# Upload data


In [ ]:
# Define correction function based on Patrignani et al. (2022) for Hydrosense II hand-held sensor


In [ ]:
# Apply correction function


### Create GeoDataframe
We will now convert the Pandas Dataframe into a GeoPandas GeoDataframe.

In [ ]:
# Convert geographic coordinates to projected coordinates 
# We will use UTM-14 for Kansas (https://epsg.io/32614)


In [ ]:
# Approximate field boundary using the convex hull

# Alternatively you can read your field boundary in geojson, KML, or shapefile format
# field_bnd = gpd.read_file('sm_data/neon_kona_bnd.geojson').to_crs(crs=32614)


In [ ]:
# Display all field points and the field boundary


### Create field grid
Now need to generate a raster layer that spans the entire field, with each grid cell containing a value estimated from nearby point observations.

In [ ]:
# Get bounds considering all points (note that bounds is for each geometry, while total bounds is for the entire GeoDataFrame)


In [ ]:
# Create grid


In [ ]:
# Create raster using xarray


In [ ]:
# Create a GeoDataframe with the centroids, so that we can use them to find cells inside and outside of the field boundary


In [ ]:
# Overlay all vector (sampling points ) and raster data


### Create mask
The mask represents the grid cells that are outside of the field boundary and that therefore we want to ignore during interpolation and the overall data analysis.

In [ ]:
# Find grid points inside of field boundary

# Convert the GeoSeries (1D column) to matrix form (2d array)

# Convert array into Xarray


In [ ]:
# Plot


### Interpolate soil moisture for each survey
Here we will interpolate point values collected during each survey to the regular grid. There will be one grid per survey.


In [ ]:
# Create a space-time DataArray with all the interpolated surveys


In [ ]:
# Plot particular survey


In [ ]:
# Plot all of them


In [ ]:
# Compute mean relative diffences


In [ ]:
# Plot


### Determine management zones based on soil moisture

In [ ]:
# To do by students following NDVI example